<a href="https://colab.research.google.com/github/gacerioni/redis-workshop-json-search-vs/blob/master/redis-workshop-vector-llm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Workshop - Redis como VectorDB e RAG
## Vector Searches & Large Language Models

![Redis](https://redis.io/wp-content/uploads/2024/04/Logotype.svg?auto=webp&quality=85,75&width=120)


Bem-vind[ao]s ao Workshop! Vamos ter uma experiência hands-on sobre alguns temas centrais do Redis, bem além do Caching.


Para uma experiência premium, como a que eu quero que vocês tenham, recomendo MUITO utilizar o Redis Insight (App ou Web) pra apoiar na visualização dos dados.

https://redis.com/redis-enterprise/redis-insight/

---

Novamente, vamos direto ao ponto. Para pegar o fio da meada, passando pela introdução, veja este outro notebook [aqui](https://colab.research.google.com/github/gacerioni/redis-workshop-json-search-vs/blob/master/redis-workshop-vector-similarity-search.ipynb).

---

## Objetivos do Workshop

Vamos tornar tangível um dos meus casos de uso favoritos com Redis e ChatGPT: RAG.

Neste workshop, vamos usar o Redis como um cérebro extra, estilo cyberpunk, para o ChatGPT.

Durante uma interação com o chat llm, vamos passar documentos do Redis como contexto privado e atualizado para o ChatGPT. Vamos ver juntos.

## Mas o que é RAG mesmo?

**Retrieval-Augmented Generation** (RAG) é uma técnica poderosa no campo do Processamento de Linguagem Natural (NLP) que combina os pontos fortes dos modelos de linguagem pré-treinados com os benefícios dos sistemas de recuperação de informação ultra performáticos, como o nosso amigo Redis aqui.

Em outras palavras, o RAG utiliza a busca semântica para encontrar informações relevantes e, em seguida, usa um modelo de linguagem para gerar respostas mais precisas e contextualmente relevantes para um usuário curioso, como nós, através de prompts.

No contexto do Redis, isso significa usar o Redis como um banco de dados de vetores para armazenar e recuperar informações de forma eficiente. O Redis facilita a comparação de vetores para determinar similaridades, permitindo buscas rápidas e precisas.

**Quando aplicado ao RAG, o Redis pode melhorar significativamente a precisão e relevância das respostas geradas por modelos de linguagem, reduzindo alucinações e fornecendo informações atualizadas. Sabe? Quando o ChatGPT fala com convicção algo completamente fora da caixinha? 😆**




Espero que gostem! 🖖

# Setup Rápido

## Instalaçao das libs do Python e redis-cli

In [6]:
# Instale o Redis client e tambem o Hugging Face sentence transformers, pois vamos gerar os vetores aqui mesmo
!pip install -q redis sentence_transformers

# Instale tbm algumas libs pra gente brincar com LLM
# este comando baixa umas paradas de 700MB+, ok? Leva dois minutinhos.
!pip install -r https://raw.githubusercontent.com/gacerioni/redis-workshop-json-search-vs/master/deps/llm-movies/requirements.txt

# E instalar a CLI, via redis-tools, que inclui a famosa redis-cli
!apt-get update
!apt-get install -y redis-tools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.0/172.0 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.6/388.6 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.2/133.2 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 https://packages.redis.io/deb jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
^C


## Deployment do Redis Stack - Um passo importante para esta demo em específico

Pessoal, esse Workshop de VectorSearch vai passar de 30MB.

---

**Ou seja: pra gente poder fazer esse e os futuros de Vector com LLM, vamos rodar o `redis-stack` aqui, locamente, neste notebook mesmo.**

---

Para isso, basta executar:

In [2]:
%%sh
curl -fsSL https://packages.redis.io/gpg | sudo gpg --dearmor -o /usr/share/keyrings/redis-archive-keyring.gpg
echo "deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb $(lsb_release -cs) main" | sudo tee /etc/apt/sources.list.d/redis.list
sudo apt-get update  > /dev/null 2>&1
sudo apt-get install redis-stack-server  > /dev/null 2>&1
redis-stack-server --daemonize yes

deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb jammy main
Starting redis-stack-server, database path /var/lib/redis-stack


gpg: cannot open '/dev/tty': No such device or address
curl: (23) Failed writing body


### Conectando com o Redis server

In [4]:
import os

# Coloque aqui os dados do seu DB do Redis Cloud
REDIS_HOST="localhost"
REDIS_PORT=6379
REDIS_PASSWORD=""

# Caso o SSL esteja ativo pro endpoint, adicione --tls
# Recomendo não misturar lé com cré aqui, visto que não vamos ter nenhuma informação sensível passando pelo fio.
if REDIS_PASSWORD!="":
  os.environ["REDIS_CONN"]=f"-h {REDIS_HOST} -p {REDIS_PORT} -a {REDIS_PASSWORD} --no-auth-warning"
else:
  os.environ["REDIS_CONN"]=f"-h {REDIS_HOST} -p {REDIS_PORT}"

# Caso o SSL esteja ativo pro endpoint, use rediss:// como o URL prefix
REDIS_URL = f"redis://:{REDIS_PASSWORD}@{REDIS_HOST}:{REDIS_PORT}"
INDEX_NAME = f"qna:idx"

# Teste a Redis connection
!redis-cli $REDIS_CONN PING

PONG


In [5]:
# Testando via Python (redis-py)
import redis
redis = redis.Redis(
  host=REDIS_HOST,
  port=REDIS_PORT,
  password=REDIS_PASSWORD)
redis.ping()

True

## Importando e preparando as libs que iremos usar

Este primeiro bloco vai garantir que todas as dependências estejam prontas pra gente brincar com o ChatGPT usando o Redis como um cérebro extra e atualizado.

O bloco de variáveis ali é basicamente eu explicando algumas preferências minhas pro Vector Search, como quantas dimensões meus vetores possuem. 384, neste caso.

In [10]:
import redis
import csv
import os
import numpy as np
from sentence_transformers import *
from redis.commands.search.query import Query
from redis.commands.search.field import TextField, TagField, VectorField
from redis.commands.search.indexDefinition import IndexDefinition, IndexType
import openai
import tiktoken


redis = redis.Redis(
  host=REDIS_HOST,
  port=REDIS_PORT,
  password=REDIS_PASSWORD,
  decode_responses=True)

# Preferencias acerca do nosso Vector Search e como o Redis deve indexar os embeddings
VSS_INDEX_TYPE = "HNSW"
VSS_DATA_TYPE = "FLOAT32"
VSS_DISTANCE = "COSINE"
VSS_DIMENSION = 384
VSS_MINIMUM_SCORE = 2

#MAX_MOVIES = 50000


redis.ping()


True

# Iniciando os trabalhos

## Passo 1 - Carregando os filmes no Redis

Primeiro, vamos baixar o CSV do nosso GitHub, contendo um monte de filmes.\
Eu fiz um dump da base do **[IMDB](https://www.imdb.com/)**, caso estejam curiosos.






In [9]:
# Baixando
!wget https://raw.githubusercontent.com/gacerioni/gabs-chatbot-llm-gpt-redis-vector-rag-demo/main/data/movies/imdb_movies.csv

--2024-05-28 19:57:21--  https://raw.githubusercontent.com/gacerioni/gabs-chatbot-llm-gpt-redis-vector-rag-demo/main/data/movies/imdb_movies.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6713035 (6.4M) [text/plain]
Saving to: ‘imdb_movies.csv’

imdb_movies.csv     100%[===================>]   6.40M  --.-KB/s    in 0.09s   

2024-05-28 19:57:21 (74.1 MB/s) - ‘imdb_movies.csv’ saved [6713035/6713035]



In [13]:
def load():
    with open("imdb_movies.csv", encoding='utf-8') as csvf:
        csvReader = csv.DictReader(csvf)
        cnt = 0
        for row in csvReader:
            redis.json().set(f'moviebot:movie:{cnt}', '$', row)
            cnt = cnt + 1
            if (cnt > MAX_MOVIES):
                break
        print("Data was loaded into Redis!")

# Flush no DB, pra gente começar do 0
redis.flushdb()

# Carregar os dados no Redis
load()


# Contar as chaves no Redis, só pra ver se tudo foi carregado
key_count = redis.dbsize()
print(f"Total de chaves no Redis: {key_count}")

Data was loaded into Redis!
Total de chaves no Redis: 10178


E ver alguns dados, só pra ter certeza que subiram para o Redis.

Notem que a gente não criou os vetores ainda, belê?

In [18]:
redis.json().get("moviebot:movie:1")

{'names': 'Avatar: The Way of Water',
 'date_x': '12/15/2022 ',
 'score': '78.0',
 'genre': 'Science Fiction,\xa0Adventure,\xa0Action',
 'overview': 'Set more than a decade after the events of the first film, learn the story of the Sully family (Jake, Neytiri, and their kids), the trouble that follows them, the lengths they go to keep each other safe, the battles they fight to stay alive, and the tragedies they endure.',
 'crew': "Sam Worthington, Jake Sully, Zoe Saldaña, Neytiri, Sigourney Weaver, Kiri / Dr. Grace Augustine, Stephen Lang, Colonel Miles Quaritch, Kate Winslet, Ronal, Cliff Curtis, Tonowari, Joel David Moore, Norm Spellman, CCH Pounder, Mo'at, Edie Falco, General Frances Ardmore",
 'orig_title': 'Avatar: The Way of Water',
 'status': ' Released',
 'orig_lang': ' English',
 'budget_x': '460000000.0',
 'revenue': '2316794914.0',
 'country': 'AU'}

## Passo 2 - Embedding - Hora de gerar vetores!

Finalmente, vamos ver nosso banco de vetores tomando forma.

Para isso, pegaremos alguns campos de informações não estruturadas de cada filme. Vamos concatenar dados interessantes como título, gênero, equipe, pontuação e sinopse.

Com essa string concatenada, geraremos vetores que representam cada filme de maneira embutida. Esses vetores serão armazenados no Redis, permitindo buscas e análises semânticas avançadas sobre os filmes.

In [20]:
# esse comando pode levar 1-2 minutos, ok?
def create_embeddings():
    model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
    for key in redis.scan_iter(match='moviebot:movie:*'):
        print(f"creating the embedding for {key}")
        result = redis.json().get(key, "$.names", "$.overview", "$.crew", "$.score", "$.genre")
        movie = f"movie title is: {result['$.names'][0]}\n"
        movie += f"movie genre is: {result['$.genre'][0]}\n"
        movie += f"movie crew is: {result['$.crew'][0]}\n"
        movie += f"movie score is: {result['$.score'][0]}\n"
        movie += f"movie overview is: {result['$.overview'][0]}\n"
        redis.json().set(key, "$.overview_embedding", model.encode(movie).astype(np.float32).tolist())

create_embeddings()

Streaming output truncated to the last 5000 lines.
creating the embedding for moviebot:movie:8787
creating the embedding for moviebot:movie:7950
creating the embedding for moviebot:movie:1436
creating the embedding for moviebot:movie:10089
creating the embedding for moviebot:movie:3310
creating the embedding for moviebot:movie:7002
creating the embedding for moviebot:movie:8676
creating the embedding for moviebot:movie:3662
creating the embedding for moviebot:movie:6678
creating the embedding for moviebot:movie:4650
creating the embedding for moviebot:movie:9825
creating the embedding for moviebot:movie:4265
creating the embedding for moviebot:movie:7867
creating the embedding for moviebot:movie:8306
creating the embedding for moviebot:movie:434
creating the embedding for moviebot:movie:9279
creating the embedding for moviebot:movie:8243
creating the embedding for moviebot:movie:5053
creating the embedding for moviebot:movie:10120
creating the embedding for moviebot:movie:3980
creating

#### Checkpoint - vamos entender o que temos no Redis agora

Nós criamos o embedding usando outros atributos do filme, e guardamos esse "fingerprint" junto do próprio documento JSON que representa cada filme.

Olha só como ficou agora:

In [25]:
import json

movie_with_embedding = redis.json().get("moviebot:movie:1")
pretty_json = json.dumps(movie_with_embedding, indent=4, ensure_ascii=False)
print(pretty_json)

{
    "names": "Avatar: The Way of Water",
    "date_x": "12/15/2022 ",
    "score": "78.0",
    "genre": "Science Fiction, Adventure, Action",
    "overview": "Set more than a decade after the events of the first film, learn the story of the Sully family (Jake, Neytiri, and their kids), the trouble that follows them, the lengths they go to keep each other safe, the battles they fight to stay alive, and the tragedies they endure.",
    "crew": "Sam Worthington, Jake Sully, Zoe Saldaña, Neytiri, Sigourney Weaver, Kiri / Dr. Grace Augustine, Stephen Lang, Colonel Miles Quaritch, Kate Winslet, Ronal, Cliff Curtis, Tonowari, Joel David Moore, Norm Spellman, CCH Pounder, Mo'at, Edie Falco, General Frances Ardmore",
    "orig_title": "Avatar: The Way of Water",
    "status": " Released",
    "orig_lang": " English",
    "budget_x": "460000000.0",
    "revenue": "2316794914.0",
    "country": "AU",
    "overview_embedding": [
        -0.07662265002727509,
        -0.002546886680647731,
      

## Passo 3 - Criando o índex para o JSON do Filme

Vamos criar o índex, sem segredo, pra gente fazer as queries depois.

In [28]:
import time

def create_index():
    indexes = redis.execute_command("FT._LIST")
    if "movie_idx" not in indexes:
        index_def = IndexDefinition(prefix=["moviebot:movie:"], index_type=IndexType.JSON)
        schema = (TextField("$.crew", as_name="crew"),
                  TextField("$.overview", as_name="overview"),
                  TagField("$.genre", as_name="genre"),
                  TagField("$.names", as_name="names"),
                  VectorField("$.overview_embedding", VSS_INDEX_TYPE,
                              {"TYPE": VSS_DATA_TYPE, "DIM": VSS_DIMENSION, "DISTANCE_METRIC": VSS_DISTANCE},
                              as_name="embedding"))
        redis.ft('movie_idx').create_index(schema, definition=index_def)
        print("The index has been created")
    else:
        print("The index exists")

# Cria o index aqui mesmo
create_index()

The index exists


E vamos testar o index por aqui mesmo também, fazendo uma Full Text Search bem preguiçosa

In [33]:
# Testa aqui mesmo
time.sleep(2)

# Executar a busca, direto aqui mesmo
# Redis é binary safe, então esse abaixo, com ñ, precisa funfar de boa
search_result = redis.execute_command("FT.SEARCH", "movie_idx", "@crew:'Zoe Saldaña'")
print(search_result)

[26, 'moviebot:movie:68', ['$', '{"names":"Avatar","date_x":"12/17/2009 ","score":"76.0","genre":"Action,\xa0Adventure,\xa0Fantasy,\xa0Science Fiction","overview":"In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization.","crew":"Sam Worthington, Jake Sully, Zoe Saldaña, Neytiri, Sigourney Weaver, Dr. Grace Augustine, Stephen Lang, Colonel Miles Quaritch, Michelle Rodriguez, Trudy Chacon, Giovanni Ribisi, Parker Selfridge, Joel David Moore, Norm Spellman, CCH Pounder, Mo\'at, Wes Studi, Eytukan","orig_title":"Avatar","status":" Released","orig_lang":" English","budget_x":"237000000.0","revenue":"2923706026.0","country":"AU","overview_embedding":[-0.024951674044132233,0.021150358021259308,-0.009504531510174274,-0.06110683083534241,-0.06520567834377289,0.010032949037849905,-0.028909243643283844,-0.0449451245367527,0.039477039128541946,-0.0166932325810194,0.04766632989048